In [1]:
import pandas as pd
df = pd.read_csv('Bengaluru_House_Data.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [2]:
df.drop(['area_type','availability','society','balcony'],axis=1,inplace=True)
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [3]:
df['location'] = df['location'].fillna('Sarjapur  Road')

In [4]:
df['size'] = df['size'].fillna('2 BHK')

In [5]:
df['bath'] = df['bath'].fillna(df['bath'].median())

In [6]:
df['bhk'] = df['size'].str.split().str.get(0).astype(int)

In [7]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [9]:
def convertRange(x):
    temp = x.split('-')
    if(len(temp) == 2):
        return (float(temp[0])+float(temp[1]))/2;
    try:
        return float(x)
    except:
        return None

In [10]:
df['total_sqft'] = df['total_sqft'].apply(convertRange)

In [11]:
df['total_sqft'].unique()
df.head(2)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4


In [12]:
df['price_per_sqft'] = df['price']*(1e5) / df['total_sqft']

In [13]:
df.head(2)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615


In [14]:
df['location'] = df['location'].apply(lambda x : x.strip())
location_count = df['location'].value_counts()

In [15]:
location_count_less_10 = location_count[location_count <= 10]
location_count_less_10

Dairy Circle                      10
Nagappa Reddy Layout              10
Basapura                          10
1st Block Koramangala             10
Sector 1 HSR Layout               10
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: location, Length: 1053, dtype: int64

In [16]:
df['location'] = df['location'].apply(lambda x : 'other' if x in location_count_less_10 else x)
df

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,3453.0,4.0,231.00,5,6689.834926
13316,other,4 BHK,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,2 BHK,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4 BHK,4689.0,4.0,488.00,4,10407.336319


In [17]:
(df['total_sqft']/df['bhk']).describe()

count    13274.000000
mean       575.074878
std        388.205175
min          0.250000
25%        473.333333
50%        552.500000
75%        625.000000
max      26136.000000
dtype: float64

In [18]:
import numpy as np
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key , subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output , gen_df],ignore_index=True)
    return df_output

In [19]:
df = remove_outliers_sqft(df)
df.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,11488.000000,11488.000000,11488.000000,11488.000000,11488.000000
mean,1537.976810,2.605066,105.238232,2.719185,6303.086658
std,1145.700742,1.208022,138.520527,1.152969,3636.502296
min,250.000000,1.000000,8.000000,1.000000,267.829813
25%,1100.000000,2.000000,50.000000,2.000000,4285.714286
50%,1280.000000,2.000000,70.000000,3.000000,5311.170467
75%,1660.000000,3.000000,110.000000,3.000000,6849.601172
max,52272.000000,27.000000,3600.000000,27.000000,53150.000000


In [20]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location , location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk , bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk , bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices , bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [21]:
df = bhk_outlier_remover(df)
df.shape

(8040, 7)

In [22]:
df

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,6 BHK,1200.0,6.0,125.0,6,10416.666667
3,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
4,1st Block Jayanagar,7 Bedroom,930.0,4.0,85.0,7,9139.784946
...,...,...,...,...,...,...,...
11479,other,7 Bedroom,1400.0,7.0,218.0,7,15571.428571
11482,other,2 BHK,1353.0,2.0,110.0,2,8130.081301
11483,other,1 Bedroom,812.0,1.0,26.0,1,3201.970443
11486,other,4 Bedroom,1200.0,5.0,325.0,4,27083.333333


In [23]:
df.drop(['size','price_per_sqft'],axis=1,inplace=True)
df

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1200.0,6.0,125.0,6
3,1st Block Jayanagar,1875.0,2.0,235.0,3
4,1st Block Jayanagar,930.0,4.0,85.0,7
...,...,...,...,...,...
11479,other,1400.0,7.0,218.0,7
11482,other,1353.0,2.0,110.0,2
11483,other,812.0,1.0,26.0,1
11486,other,1200.0,5.0,325.0,4


In [24]:
df.to_csv('Cleaned_bangalore_house_data.csv')

## Training Data

In [25]:
from sklearn.model_selection import train_test_split
X = df.drop(['price'],axis=1)
y = df['price']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [26]:
print(X_train.shape)
print(X_test.shape)

(5628, 4)
(2412, 4)


In [27]:
from sklearn.linear_model import LinearRegression
# OneHotEncoder : Converts categorical to binary
# StandardScaler: for standardizing numerical features by removing the mean and scaling to unit variance
from sklearn.preprocessing import OneHotEncoder , StandardScaler
# make_column_transformer : Can make different transfomrations on a column using this
from sklearn.compose import make_column_transformer
# make_pipeline : Creates a pipeline by chaining multiple transformers and estimators together 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [28]:
column_trans = make_column_transformer((OneHotEncoder(sparse=False),['location']),remainder='passthrough')

# OneHotEncoder(sparse=False): It performs one-hot encoding on categorical data, converting to a binary vector representation. 
# The sparse=False parameter ensures that the output is returned as a dense array instead of a sparse matrix.


# ['location']: This is a list that specifies the column(s) to which the first transformer should be applied.

# remainder='passthrough': This parameter indicates that any remaining columns in the dataset should be included in the 
# output as they are, without applying any further transformations. 

In [29]:
scaler = StandardScaler()

In [30]:
lr = LinearRegression()

In [31]:
# Make several transformations on the lr model
pipe = make_pipeline(column_trans , scaler , lr)

In [32]:
pipe.fit(X_train , y_train)

C:\Users\Satwik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [33]:
y_pred = pipe.predict(X_test)

In [36]:
print(r2_score(y_test,y_pred))

0.7068799086416702


In [37]:
import pickle as pickle
pickle.dump(pipe , open('LRModel.pkl','wb'))

In [38]:
# In this , we have a scikit-learn pipeline object named pipe that you want to save as a pickle file named 'LRModel.pkl'.

# pickle.dump(obj, file): This function from the pickle module is used to serialize the object obj and save it to the file
# specified by file. The obj is the pipeline object pipe that you want to save, and the file is opened in write
# binary mode ('wb').

# open('LRModel.pkl', 'wb'): This code opens the file named 'LRModel.pkl' in write binary mode ('wb'). The wb mode ensures 
# that the file is opened for writing in binary mode, which is required when saving scikit-learn objects using pickle.

In [39]:
df

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1200.0,6.0,125.0,6
3,1st Block Jayanagar,1875.0,2.0,235.0,3
4,1st Block Jayanagar,930.0,4.0,85.0,7
...,...,...,...,...,...
11479,other,1400.0,7.0,218.0,7
11482,other,1353.0,2.0,110.0,2
11483,other,812.0,1.0,26.0,1
11486,other,1200.0,5.0,325.0,4
